## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-24-02-25-30 +0000,nyt,"Trump Says U.S. ‘Armada’ Is Heading to Iran, R...",https://www.nytimes.com/2026/01/23/world/middl...
1,2026-01-24-02-14-18 +0000,nyt,Pepper-Sprayed While Pinned Down: A Searing Sc...,https://www.nytimes.com/2026/01/23/us/minneapo...
2,2026-01-24-02-07-33 +0000,nypost,Meta pauses teen access to AI characters weeks...,https://nypost.com/2026/01/23/business/meta-pa...
3,2026-01-24-02-03-13 +0000,nyt,What’s It Like to Climb a Skyscraper?,https://www.nytimes.com/2026/01/23/world/asia/...
4,2026-01-24-02-00-00 +0000,wsj,"With the Fate of Greenland in Flux, It’s Back ...",https://www.wsj.com/world/with-the-fate-of-gre...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2387/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,61
47,ice,22
30,greenland,17
516,new,14
58,tiktok,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
311,2026-01-23-03-44-07 +0000,nypost,Trump withdraws Canadian PM Mark Carney’s Boar...,https://nypost.com/2026/01/22/us-news/trump-wi...,105
200,2026-01-23-16-09-21 +0000,nyt,Trump Withdraws Carney’s Invitation for Canada...,https://www.nytimes.com/2026/01/23/us/politics...,105
266,2026-01-23-10-46-56 +0000,nyt,"After Trump’s Ultimatum, Greenland Talks Inclu...",https://www.nytimes.com/2026/01/22/world/europ...,104
7,2026-01-24-02-00-00 +0000,wsj,Trump’s aggressive deployment of ICE has thrus...,https://www.wsj.com/politics/policy/democrat-r...,103
185,2026-01-23-17-00-09 +0000,latimes,Trump lawyers urge Supreme Court to block Cali...,https://www.latimes.com/politics/story/2026-01...,102


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
311,105,2026-01-23-03-44-07 +0000,nypost,Trump withdraws Canadian PM Mark Carney’s Boar...,https://nypost.com/2026/01/22/us-news/trump-wi...
59,57,2026-01-23-23-01-19 +0000,wapo,Thousands march in Minnesota and hundreds of b...,https://www.washingtonpost.com/nation/2026/01/...
289,50,2026-01-23-08-08-54 +0000,bbc,Is the US-TikTok deal a new reality for China'...,https://www.bbc.com/news/articles/c6200mz054yo...
90,49,2026-01-23-21-53-22 +0000,nypost,Ex-Olympic snowboarder Ryan Wedding back on US...,https://nypost.com/2026/01/23/us-news/ex-olymp...
266,43,2026-01-23-10-46-56 +0000,nyt,"After Trump’s Ultimatum, Greenland Talks Inclu...",https://www.nytimes.com/2026/01/22/world/europ...
99,39,2026-01-23-21-39-00 +0000,wsj,Intel Shares Slide as Costs Pile Up in Bid to ...,https://www.wsj.com/business/earnings/intel-in...
45,38,2026-01-23-23-34-51 +0000,nypost,Ex-NJ Gov. Phil Murphy granted clemency to mom...,https://nypost.com/2026/01/23/us-news/murphy-g...
10,35,2026-01-24-01-39-26 +0000,nypost,Shocking question 4-year-old daughter of fetis...,https://nypost.com/2026/01/23/us-news/shocking...
315,31,2026-01-23-03-33-15 +0000,nypost,Gavin Newsom roasted as photo of him posing wi...,https://nypost.com/2026/01/22/us-news/gavin-ne...
297,30,2026-01-23-06-27-16 +0000,latimes,Police run over man amid gunfight on Californi...,https://www.latimes.com/california/story/2026-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
